In [40]:
import os
import re
import string
import pandas as pd
import urllib.parse

In [41]:
# Configuración de rutas
core_data_path = "core_data"
output_path = "docs"

# Leer los archivos TSV
colaboradores_tsv = os.path.join(core_data_path, "Diccionario_angl_tec_mx_private - Colaboradores.tsv")
terminos_tsv = os.path.join(core_data_path, "Diccionario_angl_tec_mx_private - Términos.tsv")

In [46]:
colaboradores_df = pd.read_csv(colaboradores_tsv, sep='\t')
terminos_df = pd.read_csv(terminos_tsv, sep='\t')

In [48]:
# Función para limpiar y generar nombres de archivo seguros
def generar_nombre_archivo(termino):
    return re.sub(r'[^a-zA-Z0-9\-]', '', termino.lower().replace(' ', '-'))

# Función para limpiar nombres de categorías y subcategorías para URLs
def generar_nombre_categoria(nombre):
    if pd.isna(nombre):
        return ''
    return re.sub(r'\s+', '-', str(nombre).lower())

# Función para crear hipervínculos de términos relacionados
def generar_hipervinculos(terminos, tipo, categoria=None, subcategoria=None):
    links = []
    for termino in terminos:
        safe_name = generar_nombre_archivo(termino)
        if tipo == "alfabeticamente":
            letra_inicial = termino[0].upper()
            link = f"[{termino}](https://maleniski.github.io/diccionario-angl-tec-mx/docs/alfabeticamente/{letra_inicial}/{safe_name}.html)"
        elif tipo == "taxonomia":
            if subcategoria:
                categoria_path = f"{generar_nombre_categoria(categoria)}/{generar_nombre_categoria(subcategoria)}"
            else:
                categoria_path = generar_nombre_categoria(categoria)
            link = f"[{termino}](https://maleniski.github.io/diccionario-angl-tec-mx/docs/{categoria_path}/{safe_name}.html)"
        links.append(link)
    return ', '.join(links)

# Crear archivo index.md en la raíz de "alfabeticamente" y las categorías principales
def crear_index_general(ruta, titulo, orden=None, parent=None, grand_parent=None):
    index_md = f"""---
layout: default
title: {titulo}
has_children: true
"""
    if orden is not None:
        index_md += f"nav_order: {orden}\n"
    if parent:
        index_md += f"parent: {parent}\n"
    if grand_parent:
        index_md += f"grand_parent: {grand_parent}\n"
    index_md += "---"
    
    with open(os.path.join(ruta, 'index.md'), 'w') as f:
        f.write(index_md)

# Crear directorios principales
alfabeticamente_path = os.path.join("docs", "alfabeticamente")

# Mover las categorías principales de "taxonomia" al root
output_path = "docs"

os.makedirs(alfabeticamente_path, exist_ok=True)

# Crear los archivos index.md en las carpetas principales con nav_order ajustado
crear_index_general(alfabeticamente_path, "Alfabéticamente", 2)

# Inicializar contadores de nav_order para las categorías
nav_order_categoria = 3

# Crear directorios y archivos Markdown
for _, row in terminos_df.iterrows():
    termino = row['Término']
    definicion = row['Definición']
    categoria = row['Categoría'] if pd.notna(row['Categoría']) else ''
    subcategoria = row['Subcategoría'] if pd.notna(row['Subcategoría']) else ''
    relacionados = row['Relacionados'].split(', ') if pd.notna(row['Relacionados']) else []

    # Crear carpeta alfabética si no existe
    letra_inicial = termino[0].upper()
    alphabet_folder = os.path.join(alfabeticamente_path, letra_inicial)
    os.makedirs(alphabet_folder, exist_ok=True)

    if letra_inicial not in nav_order_alfabetico:
        nav_order_alfabetico[letra_inicial] = len(nav_order_alfabetico) + 1
        crear_index_general(alphabet_folder, letra_inicial, nav_order_alfabetico[letra_inicial], parent="Alfabéticamente")

    # Crear contenido del archivo Markdown para el término en la carpeta alfabética
    contenido_md_alfabetico = f"""---
layout: default
title: {termino}
nav_order: {nav_order_alfabetico[letra_inicial] + len(os.listdir(alphabet_folder)) - 1}
parent: {letra_inicial}
grand_parent: Alfabéticamente
---

# {termino}

**Definición:**  
{definicion}

**Categoría:**  
{categoria+"  "}

{"**Subcategoría:**  \n" + subcategoria if subcategoria else "  \n"}

**Términos relacionados:**  
{generar_hipervinculos(relacionados, 'alfabeticamente')}
"""

    # Guardar archivo Markdown en la carpeta correspondiente
    with open(os.path.join(alphabet_folder, f'{generar_nombre_archivo(termino)}.md'), 'w') as f:
        f.write(contenido_md_alfabetico)

    # Crear la estructura de carpetas y archivos para categorías y subcategorías
    if categoria:
        categoria_folder = os.path.join(output_path, generar_nombre_categoria(categoria))
        if not os.path.exists(categoria_folder):
            os.makedirs(categoria_folder, exist_ok=True)
            crear_index_general(categoria_folder, categoria, nav_order_categoria)
            nav_order_categoria += 1

        if subcategoria:
            subcategoria_folder = os.path.join(categoria_folder, generar_nombre_categoria(subcategoria))
            os.makedirs(subcategoria_folder, exist_ok=True)
            crear_index_general(subcategoria_folder, subcategoria, None, parent=categoria)

        contenido_md_categoria = f"""---
layout: default
title: {termino}
parent: {subcategoria if subcategoria else categoria}
grand_parent: {categoria}
---

# {termino}

**Definición:**  
{definicion}

**Categoría:**  
{categoria+"  "}

{"**Subcategoría:**  \n" + subcategoria if subcategoria else "  \n"}

**Términos relacionados:**  
{generar_hipervinculos(relacionados, 'taxonomia', categoria, subcategoria)}
"""

        # Guardar archivo Markdown en la carpeta correspondiente
        with open(os.path.join(subcategoria_folder if subcategoria else categoria_folder, f'{generar_nombre_archivo(termino)}.md'), 'w') as f:
            f.write(contenido_md_categoria)